In [6]:
import os

In [7]:
%pwd

'c:\\Users\\adminn\\Documents\\GitHub\\Computer_vision_deployement'

In [3]:
os.chdir('../')

In [54]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/MohamedHmamouch/Computer_vision_deployement.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="MohamedHmamouch"
os.environ["MLFLOW_TRACKING_PASSWORD"]="200e4b749b0f8260522a65d396b66d9b385e1fd2"


In [57]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/MohamedHmamouch/Computer_vision_deployement.mlflow")


In [58]:
import tensorflow as tf

In [59]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [60]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    training_data:Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size:int


In [61]:
from CnnClassifier.constants import *
from CnnClassifier.utils.common import read_yaml, create_directories,save_json

In [62]:
class ConfigurationManager:

    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self)-> EvaluationConfig:

        eval_config=EvaluationConfig(
            path_to_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data/train",
            mlflow_uri="https://dagshub.com/MohamedHmamouch/Computer_vision_deployement.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [63]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


In [64]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [65]:
try:
    config=ConfigurationManager()
    eval_config=config.get_evaluation_config()
    evaluation=Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-01-05 23:30:05,266: INFO: common: yaml file: config\config.yaml load successfully]
[2024-01-05 23:30:05,271: INFO: common: yaml file: params.yaml load successfully]
[2024-01-05 23:30:05,273: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 27s 4s/step - loss: 26.7271 - accuracy: 0.5686
[2024-01-05 23:30:33,507: INFO: common: json file saved at scores.json]
[2024-01-05 23:30:55,319: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None)': /MohamedHmamouch/Computer_vision_deployement.mlflow/api/2.0/mlflow/runs/create]


2024/01/05 23:30:56 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-01-05 23:30:58,211: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\adminn\AppData\Local\Temp\tmppgn24ctu\model\data\model\assets
[2024-01-05 23:30:59,234: INFO: builder_impl: Assets written to: C:\Users\adminn\AppData\Local\Temp\tmppgn24ctu\model\data\model\assets]


2024/01/05 23:31:04 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\adminn\AppData\Local\Temp\tmppgn24ctu\model, flavor: tensorflow), fall back to return ['tensorflow==2.12.0']. Set logging level to DEBUG to see the full traceback.
Successfully registered model 'VGG16Model'.
2024/01/05 23:31:54 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
